# Crypto

## Modules


In [2]:
import re
import csv
import got3 as got
import numpy as np
import csv
import time
import random
import matplotlib.pylab as plt
from pytrends.request import TrendReq
import requests
from textblob import TextBlob
import pandas as pd
import lxml
import tweepy
from datetime import date


## Tidying

Read the table from Coin Market Cap into a dataframe. Drop asterisks in headers and also drop features I won't be using. Bitcoin price is stored starting from April 28th, 2013 but to make things nicer I kept only observartions starting on Jan 01, 2014 until December 22, 2018 (

In [4]:
# Obtainining the data from Coin Market Cap
url = "https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20130428&end=20181222" 
df = pd.read_html(url)[0]
# Do a bit of data tidying
df = df.drop(columns=['Volume', 'Market Cap'])
df = df.rename(index=str, columns={"Open*": "Open", "Close**": "Close"})
df = df[~df.Date.str.contains("2013")]
df['Date'] = pd.to_datetime(df.Date)
df.to_csv('Bitcoin.csv', index=False)

In [4]:
access_token = '952398350946619392-pb8XT2uxcq9rva7JaOC8N1EcQ1b1bY6'
access_token_secret = 'OmmiXrkce0QBseHrGYKgn1nT4X3bdcRRq4HnjKLKFjNZY'
consumer_key = '5bR78CaG4gfDGSYqiNNAM9dqn' 
consumer_secret = 'dHxBj6zSui09lvscnA7VXpJa7BvJ7vv3ONEJhSUYDPvvNVGsuR'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)
tweetCriteria = got.manager.TweetCriteria().setQuerySearch('Bitcoin').setSince("2018-12-10").setUntil("2018-12-15")
search_results = got.manager.TweetManager.getTweets(tweetCriteria)
csvFile = open('Tweets.csv', 'a+')

#Use csv writer
csvWriter = csv.writer(csvFile)
csvWriter.writerow(["Date", "Tweet", "Favorites", "Retweets","Sentiment"])
for tweet in search_results:
    text = tweet.text
    analysis = TextBlob(text)
    sentiment = analysis.sentiment.polarity
    csvWriter.writerow([tweet.date, tweet.text, tweet.favorites, tweet.retweets, sentiment])
csvFile.close()

In [ ]:
pytrends = TrendReq(hl='en-US', tz=360)
kw_list = ["Bitcoin"] 
pytrends.build_payload(kw_list, cat=0, timeframe='today 5-y', geo='', gprop='')
dfs = pytrends.get_historical_interest(kw_list, year_start=2018, month_start=1, day_start=1, hour_start=0, year_end=2018, month_end=12, day_end=20, cat=0, geo='', gprop='', sleep=0)
dfs